<a href="https://colab.research.google.com/github/junhojjangjjangman/NLP/blob/main/%EC%B9%B4%ED%83%80%EB%A5%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install konlpy
#!pip install sentence-transformers datasets
import nltk # 텍스트 데이터를 처리
import numpy as np # 말뭉치를 배열로 표현
import random 
import operator
import string # 표준 파이썬 문자열을 처리
from sklearn.metrics.pairwise import cosine_similarity # 이를 나중에 사용하여 두 개의 문장이 얼마나 비슷한지를 결정합니다.
from sklearn.feature_extraction.text import TfidfVectorizer # Experience 2에서 단어 가방을 만드는 함수를 만들었던 것을 기억하십니까? 이 함수는 같은 일을 합니다!

from sentence_transformers import SentenceTransformer #사전학습 모델을 가져오는 것
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
# os.getcwd() 현재 파일의 경로를 나타냄
# string에 +로 연결해서 사용 가능
filepath='/content/drive/MyDrive/Colab Notebooks/주피터/모듈 27/ans.txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data_ans=corpus.read()
# print (raw_data_ans)
filepath='/content/drive/MyDrive/Colab Notebooks/주피터/모듈 27/que.txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data=corpus.read()
# print (raw_data)


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
sent_tokens = nltk.sent_tokenize(raw_data)# 문서를 문장 목록으로 변환


df = pd.DataFrame(columns = ['Que','Ans'])
df['Que'] = sent_tokens
df

,Que,Ans
0,카타르 월드컵이 열리는 카타르란 나라의 수도는 어디인가요?,NaN
1,월드컵은 몇 개의 나라가 참가하나요?,NaN
2,월드컵이 열리는 카타르는 어느 대륙에 속한 나라인가요?,NaN
3,카타르 월드컵 한국 예선 일정은 어떻게 되나요?,NaN
4,결승전은 언제 하나요?,NaN
5,준결승은 언제 하나요?,NaN


In [ ]:
sent_tokens_ans = nltk.sent_tokenize(raw_data_ans)# 문서를 문장 목록으로 변환
sent_tokens_ans
df['Ans'] = sent_tokens_ans
df

,Que,Ans
0,카타르 월드컵이 열리는 카타르란 나라의 수도는 어디인가요?,카타르의 수도는 도하 라는 도시이며 사실상 카타르라는 나라가 워낙 작기 때문에 카타...
1,월드컵은 몇 개의 나라가 참가하나요?,월드컵 본선은 이번 카타르 월드컵까지 총 32개국의 국가가 출전하여 4개국 총 8개...
2,월드컵이 열리는 카타르는 어느 대륙에 속한 나라인가요?,카타르는 서 아시아에 속한 나라로 아랍어를 사용하며 사우디아라비아에 근접한 나라입니다.
3,카타르 월드컵 한국 예선 일정은 어떻게 되나요?,11월 24일 목요일 22시에 우루과이 11월 28일 월요일 22시에 가나 12월 ...
4,결승전은 언제 하나요?,12월 19일 12시에 결승전을 진행합니다.
5,준결승은 언제 하나요?,준결승 첫 경기는 12월 14일 04시에 진행합니다 두 번째 경기는 12월 15일 ...


### sentence-BERT 모델 가져오기

In [ ]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask') # embedding을 하기위해 encoder를 가져온 것.

- jhgan/ko-sroberta-multitask 모델  
- 한국어 문장 임베딩에 활용하기 위해 만들어진 한국어 사전학습 모델  
- KorNLU 데이터셋으로 한국어 사전학습 모델을 파인튜닝하고, 파인튜닝된 모델을 sentence-transformers에서 다운로드받아 활용할 수 있다.

In [ ]:
sentences=["안녕하세요?", "테스트를 위한 문장입니다."]
embeddings=model.encode(sentences)
print(embeddings)

[[-0.37510458 -0.7733841   0.5927712  ...  0.57923526  0.32683477
  -0.6508966 ]
 [-0.1708429  -0.6179124  -0.53709275 ...  0.377429    0.7510764
  -0.19669944]]


In [ ]:
df['embedding']=pd.Series([[]]*len(df)) # 길이를 맞춰주기 위해서 곱함
df['embedding']=df['Que'].map(lambda x : list(model.encode(x)))
df.head()

,Que,Ans,embedding,distance
0,카타르 월드컵이 열리는 카타르란 나라의 수도는 어디인가요?,카타르의 수도는 도하 라는 도시이며 사실상 카타르라는 나라가 워낙 작기 때문에 카타...,"[-0.16749455, 0.021784995, -0.20204595, -0.015...",0.170786
1,월드컵은 몇 개의 나라가 참가하나요?,월드컵 본선은 이번 카타르 월드컵까지 총 32개국의 국가가 출전하여 4개국 총 8개...,"[-0.44979966, -0.06767484, -0.010474164, -0.52...",0.218278
2,월드컵이 열리는 카타르는 어느 대륙에 속한 나라인가요?,카타르는 서 아시아에 속한 나라로 아랍어를 사용하며 사우디아라비아에 근접한 나라입니다.,"[0.017104436, 0.016655924, -0.10368456, 0.2016...",0.185610
3,카타르 월드컵 한국 예선 일정은 어떻게 되나요?,11월 24일 목요일 22시에 우루과이 11월 28일 월요일 22시에 가나 12월 ...,"[-0.6807921, 0.5258516, -0.0603975, -0.7469155...",0.292740
4,결승전은 언제 하나요?,12월 19일 12시에 결승전을 진행합니다.,"[-0.40630546, 0.5748352, -0.16056097, -0.67386...",0.634095


In [ ]:
# test_text='결승전 날짜'
# ebd=model.encode(test_text)

In [ ]:
def answer():
  df['distance']=df['embedding'].map(lambda x : cosine_similarity([ebd], [x]).squeeze())#차원을 없애주기 위해서 squeeze를 씀.
  df.head()

  answer1 = df.loc[df['distance'].idxmax()]
  answer2 = df.loc[df['distance'].idxmin()]

  print('유사한 질문 :', answer1['Que'])
  print('챗봇 답변 :', answer1['Ans'])
  print('유사도 :', answer1['distance'])
  print('\n\n')
  print('유사한 질문 :', answer2['Que'])
  print('챗봇 답변 :', answer2['Ans'])
  print('유사도 :', answer2['distance'])

In [ ]:
while True:
  test_text=input()
  if test_text == 'q':
    break
  ebd=model.encode(test_text)
  answer()
  print()


수도는 어디인가요
유사한 질문 : 카타르 월드컵이 열리는 카타르란 나라의 수도는 어디인가요?
챗봇 답변 : 카타르의 수도는 도하 라는 도시이며 사실상 카타르라는 나라가 워낙 작기 때문에 카타르가 곧 도하고 도하가 곧 카타르라고 생각 하시면 됩니다.
유사도 : 0.3721769452095032



유사한 질문 : 카타르 월드컵 한국 예선 일정은 어떻게 되나요?
챗봇 답변 : 11월 24일 목요일 22시에 우루과이 11월 28일 월요일 22시에 가나 12월 3일 토요일 12시에 포르투칼.
유사도 : 0.28167736530303955

ㅂ
유사한 질문 : 준결승은 언제 하나요?
챗봇 답변 : 준결승 첫 경기는 12월 14일 04시에 진행합니다 두 번째 경기는 12월 15일 04시에 진행합니다.
유사도 : 0.2867169976234436



유사한 질문 : 카타르 월드컵이 열리는 카타르란 나라의 수도는 어디인가요?
챗봇 답변 : 카타르의 수도는 도하 라는 도시이며 사실상 카타르라는 나라가 워낙 작기 때문에 카타르가 곧 도하고 도하가 곧 카타르라고 생각 하시면 됩니다.
유사도 : -0.020367605611681938

q
